In [2]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [3]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [4]:
import en_core_web_lg
nlp_lg=en_core_web_lg.load()
import joblib as jlb
jlb.dump(nlp_lg,"large_en_model.sav")

['large_en_model.sav']

In [5]:
from sklearn.preprocessing import MinMaxScaler


In [108]:
from imblearn.over_sampling import SMOTE


In [6]:
df=pd.read_excel("Text_Mining_Reviews.xlsx",sheet_name="Data")
df

,Number,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score
0,0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,682.0,37.44,negative
1,1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,712.0,31.82,positive
2,2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,538.0,26.21,negative
3,3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,1354.0,59.90,negative
4,4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,2331.0,119.50,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,3454.0,167.86,NaN
16655,16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,1456.0,73.63,NaN
16656,16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,872.0,41.18,NaN
16657,16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,877.0,41.50,NaN


In [7]:
df=df.drop(['Number'], axis=1)

In [8]:
l=[]
for i in range(0,len(df.index)):
    s=str(df.iloc[i][0])+" "+str(df.iloc[i][3])
    l.append(s)

In [9]:
# from nltk.tokenize import RegexpTokenizer
# from nltk.stem.porter import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
nltk.download('stopwords')
tokenizer=RegexpTokenizer(r'\w+')
en_stopwords=set(stopwords.words('english'))
ps=PorterStemmer()
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
def getCleanedText(text):
    text=str(text)
    text=text.lower()
    text = re.sub('[^A-Za-z]', ' ', text)
#     text=[x for x in text if x.lower() in words]
    tokens=tokenizer.tokenize(text)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    stemmed_tokens=[ps.stem(tokens) for tokens in new_tokens]
    final=[x for x in stemmed_tokens if x.isdigit()==False ]
    final=[x for x in final if x.lower() in words and len(x)>2]
    clean_text=" ".join(final)
    return clean_text

In [11]:
getCleanedText(l[0])
l=[getCleanedText(i) for i in l]

In [12]:
df['clean_text']=l

In [13]:
df["binary_score"]=df["binary_score"].replace({"positive":1,"negative":0})

In [14]:
def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

In [15]:
df['subjectivity']=df['clean_text'].apply(get_subjectivity)
df['polarity']=df['clean_text'].apply(get_polarity)


In [16]:
def getSIA(text):
    sia=SentimentIntensityAnalyzer()
    sentiment=sia.polarity_scores(text)
    return sentiment

In [17]:
compound=[]
neg=[]
pos=[]
neu=[]
SIA=0
for i in range(0,len(df['clean_text'])):
    SIA=getSIA(df['clean_text'][i])
    compound.append(SIA['compound'])
    neg.append(SIA['neg'])
    pos.append(SIA['pos'])
    neu.append(SIA['neu'])

In [18]:
df['compound']=compound
df['negative']=neg
df['positive']=pos
df['neutral']=neu

In [19]:
df

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,responses_message_length,gunning_fog_index,binary_score,clean_text,subjectivity,polarity,compound,negative,positive,neutral
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,682.0,37.44,0.0,overpay took loan breed term lie soften result...,0.288095,-0.066667,-0.5358,0.116,0.000,0.884
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,712.0,31.82,1.0,great card year card interest free period mont...,0.631250,0.466667,0.9738,0.084,0.457,0.459
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,538.0,26.21,0.0,call third would like ask question right bank ...,0.267143,0.097143,-0.2732,0.165,0.062,0.773
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,1354.0,59.90,0.0,claim connect loan amount bank contract way co...,0.100000,0.300000,0.8779,0.126,0.286,0.588
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,2331.0,119.50,0.0,credit fraud husband need money went nearest s...,0.406944,-0.016865,0.6555,0.117,0.173,0.710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,3454.0,167.86,NaN,take care money cooper bank seem good even per...,0.386806,0.073264,0.9758,0.069,0.176,0.755
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,1456.0,73.63,NaN,card lock may made sever transact amount sever...,0.434972,-0.070223,-0.7234,0.187,0.146,0.667
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,872.0,41.18,NaN,good afternoon name client bank moment delay d...,0.550000,0.418182,-0.4464,0.185,0.094,0.722
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,877.0,41.50,NaN,mislead enter loan agreement loan bank told lo...,0.500000,0.416667,0.7964,0.057,0.227,0.716


In [20]:
df.iloc[0][18]

0.0

In [21]:
ll=[]

for i in range(0,len(df.index)):
    if str(df.iloc[i][1])=='nan':
        if df.iloc[i][12]==0.0:
            if df.iloc[i][17]<0.081000:
                val=1.0
            else:
                val=2.0
        else:
            if df.iloc[i][18]<0.234000:
                val=4.0
            else:
                val=5.0  
        ll.append(val)
    else:
        ll.append(df.iloc[i][1])

In [22]:
len(ll)

16659

In [23]:
ll[0]

2.0

In [24]:
df['rating']=ll

In [25]:
df

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,...,gunning_fog_index,binary_score,clean_text,subjectivity,polarity,compound,negative,positive,neutral,rating
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,...,37.44,0.0,overpay took loan breed term lie soften result...,0.288095,-0.066667,-0.5358,0.116,0.000,0.884,2.0
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,...,31.82,1.0,great card year card interest free period mont...,0.631250,0.466667,0.9738,0.084,0.457,0.459,5.0
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,...,26.21,0.0,call third would like ask question right bank ...,0.267143,0.097143,-0.2732,0.165,0.062,0.773,1.0
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,...,59.90,0.0,claim connect loan amount bank contract way co...,0.100000,0.300000,0.8779,0.126,0.286,0.588,1.0
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,...,119.50,0.0,credit fraud husband need money went nearest s...,0.406944,-0.016865,0.6555,0.117,0.173,0.710,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,...,167.86,NaN,take care money cooper bank seem good even per...,0.386806,0.073264,0.9758,0.069,0.176,0.755,1.0
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,...,73.63,NaN,card lock may made sever transact amount sever...,0.434972,-0.070223,-0.7234,0.187,0.146,0.667,1.0
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,...,41.18,NaN,good afternoon name client bank moment delay d...,0.550000,0.418182,-0.4464,0.185,0.094,0.722,4.0
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,...,41.50,NaN,mislead enter loan agreement loan bank told lo...,0.500000,0.416667,0.7964,0.057,0.227,0.716,4.0


In [26]:
nlp_load=jlb.load("large_en_model.sav")

In [27]:
def get_vectors(doc):
    doc=nlp_load(doc)
    return doc.vector

In [28]:
x_vec=df['clean_text'].apply(lambda doc:get_vectors(doc))

In [29]:
x_vec.shape

(16659,)

In [30]:
d1 = pd.DataFrame(x_vec[0])
d1=d1.T
d1

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.322805,0.20874,-0.125365,-0.014328,-0.036668,-0.20586,-0.172317,-0.121677,0.129793,2.24883,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.05939


In [31]:
for i in range(1,len(df.index)):
    l=list(x_vec[i])
    d1.loc[i]=l

In [42]:
d2=df
df

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,...,gunning_fog_index,binary_score,clean_text,subjectivity,polarity,compound,negative,positive,neutral,rating
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,...,37.44,0.0,overpay took loan breed term lie soften result...,0.288095,-0.066667,-0.5358,0.116,0.000,0.884,2.0
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,...,31.82,1.0,great card year card interest free period mont...,0.631250,0.466667,0.9738,0.084,0.457,0.459,5.0
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,...,26.21,0.0,call third would like ask question right bank ...,0.267143,0.097143,-0.2732,0.165,0.062,0.773,1.0
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,...,59.90,0.0,claim connect loan amount bank contract way co...,0.100000,0.300000,0.8779,0.126,0.286,0.588,1.0
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,...,119.50,0.0,credit fraud husband need money went nearest s...,0.406944,-0.016865,0.6555,0.117,0.173,0.710,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,...,167.86,NaN,take care money cooper bank seem good even per...,0.386806,0.073264,0.9758,0.069,0.176,0.755,1.0
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,...,73.63,NaN,card lock may made sever transact amount sever...,0.434972,-0.070223,-0.7234,0.187,0.146,0.667,1.0
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,...,41.18,NaN,good afternoon name client bank moment delay d...,0.550000,0.418182,-0.4464,0.185,0.094,0.722,4.0
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,...,41.50,NaN,mislead enter loan agreement loan bank told lo...,0.500000,0.416667,0.7964,0.057,0.227,0.716,4.0


In [40]:
d1

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.322805,0.208740,-0.125365,-0.014328,-0.036668,-0.205860,-0.172317,-0.121677,0.129793,2.248830,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,-0.224255,0.129340,-0.069198,-0.119912,-0.002965,-0.094610,-0.012933,-0.236394,-0.022953,2.037880,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,-0.264667,0.180769,-0.244706,0.032709,0.174005,-0.169866,0.050055,-0.312749,0.036248,2.323701,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,-0.316561,0.126572,-0.219372,0.016243,-0.063551,-0.027468,-0.122361,-0.294620,0.122126,2.410608,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,-0.159453,0.170062,-0.194565,0.015306,-0.005548,-0.078206,-0.065153,-0.166629,0.058344,2.372043,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,-0.147047,0.166510,-0.230434,-0.006345,0.022306,-0.089318,-0.045631,-0.185138,0.036969,2.293782,...,-0.264311,0.155511,-0.077682,-0.083905,0.136575,-0.115191,-0.037609,-0.074105,0.029689,0.110392
16655,-0.063548,0.168612,-0.210156,-0.018253,0.079034,-0.078289,-0.034102,-0.206846,-0.023649,2.232468,...,-0.220124,0.117387,-0.020000,-0.086508,0.082196,-0.148433,-0.052222,-0.112000,0.003428,0.146342
16656,-0.102735,0.139676,-0.244953,0.017891,-0.004391,-0.077289,-0.037822,-0.184575,0.012428,2.131650,...,-0.251914,0.170144,-0.089434,-0.036835,0.155465,-0.190750,0.028117,-0.026899,-0.031707,0.058622
16657,-0.256856,0.196397,-0.304738,0.040157,0.000426,-0.108772,-0.106098,-0.315313,0.062923,2.456368,...,-0.339285,0.177451,-0.033658,-0.076011,0.196520,-0.187527,-0.016569,-0.053284,0.065623,0.041845


In [57]:
df_final=pd.concat([df,d1], axis=1)

In [58]:
df_final

,responses_header,responses_rating_grade,responses_status,responses_message,responses_datetime,reviewer,comments,views,id,have_email,...,290,291,292,293,294,295,296,297,298,299
0,Surprise from overpayment,NaN,NaN,"I took a loan of 100,000 rubles in 2015. The o...",2017-08-12 12:10:00,g*******@mail.ru,0,1169,21.0,1.0,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,"Great card, increased warranty!",5.0,Not counted,For a little more than a year I issued your ca...,2017-08-15 16:15:00,540999,0,1072,117.0,0.0,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,Calls to third parties,1.0,NaN,I would like to ask a question. By what right ...,2016-06-23 10:05:00,appetita,2,1280,131.0,0.0,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,Claim for connecting additional services,1.0,Not counted,When applying for a consumer loan in the CC at...,2017-08-16 17:58:00,Galoyan15,5,1085,156.0,0.0,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,Consumer credit insurance fraud,NaN,NaN,08/10/17 my husband received an SMS about the ...,2017-08-17 14:07:00,elenaklim,1,1556,172.0,0.0,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,Take care of your nerves and money,1.0,Not counted,"My cooperation with the bank began in 2014, ev...",2016-05-10 07:07:00,julka59,0,1454,554833.0,0.0,...,-0.264311,0.155511,-0.077682,-0.083905,0.136575,-0.115191,-0.037609,-0.074105,0.029689,0.110392
16655,Card lock,1.0,NaN,"On May 4, I made several transactions for a la...",2016-05-10 10:21:00,roboglock,4,1432,554880.0,0.0,...,-0.220124,0.117387,-0.020000,-0.086508,0.082196,-0.148433,-0.052222,-0.112000,0.003428,0.146342
16656,Quality of service,NaN,NaN,"Good afternoon! My name is Julia, I am a clien...",2016-05-10 15:49:00,J*******@mail.ru,1,1280,554956.0,0.0,...,-0.251914,0.170144,-0.089434,-0.036835,0.155465,-0.190750,0.028117,-0.026899,-0.031707,0.058622
16657,Misleading,NaN,NaN,"On 05/07/2015, I entered into a loan agreement...",2016-05-10 15:54:00,amkrutova,2,1354,554957.0,0.0,...,-0.339285,0.177451,-0.033658,-0.076011,0.196520,-0.187527,-0.016569,-0.053284,0.065623,0.041845


In [59]:
col=list(df_final.columns)

In [60]:
col

['responses_header',
 'responses_rating_grade',
 'responses_status',
 'responses_message',
 'responses_datetime',
 'reviewer',
 'comments',
 'views',
 'id',
 'have_email',
 'responses_message_length',
 'gunning_fog_index',
 'binary_score',
 'clean_text',
 'subjectivity',
 'polarity',
 'compound',
 'negative',
 'positive',
 'neutral',
 'rating',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 

In [61]:
df_final=df_final.drop(['responses_header','responses_rating_grade','responses_status','responses_message','responses_datetime','reviewer','id','clean_text'],axis=1)

In [62]:
df_final

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,...,290,291,292,293,294,295,296,297,298,299
0,0,1169,1.0,682.0,37.44,0.0,0.288095,-0.066667,-0.5358,0.116,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,0,1072,0.0,712.0,31.82,1.0,0.631250,0.466667,0.9738,0.084,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,2,1280,0.0,538.0,26.21,0.0,0.267143,0.097143,-0.2732,0.165,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,5,1085,0.0,1354.0,59.90,0.0,0.100000,0.300000,0.8779,0.126,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,1,1556,0.0,2331.0,119.50,0.0,0.406944,-0.016865,0.6555,0.117,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,0,1454,0.0,3454.0,167.86,NaN,0.386806,0.073264,0.9758,0.069,...,-0.264311,0.155511,-0.077682,-0.083905,0.136575,-0.115191,-0.037609,-0.074105,0.029689,0.110392
16655,4,1432,0.0,1456.0,73.63,NaN,0.434972,-0.070223,-0.7234,0.187,...,-0.220124,0.117387,-0.020000,-0.086508,0.082196,-0.148433,-0.052222,-0.112000,0.003428,0.146342
16656,1,1280,0.0,872.0,41.18,NaN,0.550000,0.418182,-0.4464,0.185,...,-0.251914,0.170144,-0.089434,-0.036835,0.155465,-0.190750,0.028117,-0.026899,-0.031707,0.058622
16657,2,1354,0.0,877.0,41.50,NaN,0.500000,0.416667,0.7964,0.057,...,-0.339285,0.177451,-0.033658,-0.076011,0.196520,-0.187527,-0.016569,-0.053284,0.065623,0.041845


In [63]:
# from sklearn.preprocessing import MinMaxScaler
# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
col=['comments','views','have_email','responses_message_length','gunning_fog_index','rating']
df_final[['comments','views','have_email','responses_message_length','gunning_fog_index','rating']] = pd.DataFrame(scaler.fit_transform(df_final[['comments','views','have_email','responses_message_length','gunning_fog_index','rating']]))



In [64]:
df_final

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.027590,1.0,0.041849,0.051988,0.0,0.288095,-0.066667,-0.5358,0.116,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,0.000000,0.024992,0.0,0.043935,0.043551,1.0,0.631250,0.466667,0.9738,0.084,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,0.041667,0.030564,0.0,0.031839,0.035129,0.0,0.267143,0.097143,-0.2732,0.165,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,0.104167,0.025340,0.0,0.088564,0.085705,0.0,0.100000,0.300000,0.8779,0.126,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,0.020833,0.037957,0.0,0.156482,0.175179,0.0,0.406944,-0.016865,0.6555,0.117,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,0.000000,0.035224,0.0,0.234550,0.247778,NaN,0.386806,0.073264,0.9758,0.069,...,-0.264311,0.155511,-0.077682,-0.083905,0.136575,-0.115191,-0.037609,-0.074105,0.029689,0.110392
16655,0.083333,0.034635,0.0,0.095655,0.106317,NaN,0.434972,-0.070223,-0.7234,0.187,...,-0.220124,0.117387,-0.020000,-0.086508,0.082196,-0.148433,-0.052222,-0.112000,0.003428,0.146342
16656,0.020833,0.030564,0.0,0.055057,0.057602,NaN,0.550000,0.418182,-0.4464,0.185,...,-0.251914,0.170144,-0.089434,-0.036835,0.155465,-0.190750,0.028117,-0.026899,-0.031707,0.058622
16657,0.041667,0.032546,0.0,0.055405,0.058083,NaN,0.500000,0.416667,0.7964,0.057,...,-0.339285,0.177451,-0.033658,-0.076011,0.196520,-0.187527,-0.016569,-0.053284,0.065623,0.041845


In [106]:
df_final.to_csv("rating+vader+smote+spacy.csv",index=False)

In [94]:
df_final=pd.read_csv("rating+vader+smote+spacy.csv")

In [95]:
df_final

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.027590,1.0,0.041849,0.051988,0.0,0.288095,-0.066667,-0.5358,0.116,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,0.000000,0.024992,0.0,0.043935,0.043551,1.0,0.631250,0.466667,0.9738,0.084,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,0.041667,0.030564,0.0,0.031839,0.035129,0.0,0.267143,0.097143,-0.2732,0.165,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,0.104167,0.025340,0.0,0.088564,0.085705,0.0,0.100000,0.300000,0.8779,0.126,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,0.020833,0.037957,0.0,0.156482,0.175179,0.0,0.406944,-0.016865,0.6555,0.117,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16654,0.000000,0.035224,0.0,0.234550,0.247778,NaN,0.386806,0.073264,0.9758,0.069,...,-0.264311,0.155511,-0.077682,-0.083905,0.136575,-0.115191,-0.037609,-0.074105,0.029689,0.110392
16655,0.083333,0.034635,0.0,0.095655,0.106317,NaN,0.434972,-0.070223,-0.7234,0.187,...,-0.220124,0.117387,-0.020000,-0.086508,0.082196,-0.148433,-0.052222,-0.112000,0.003428,0.146342
16656,0.020833,0.030564,0.0,0.055057,0.057602,NaN,0.550000,0.418182,-0.4464,0.185,...,-0.251914,0.170144,-0.089434,-0.036835,0.155465,-0.190750,0.028117,-0.026899,-0.031707,0.058622
16657,0.041667,0.032546,0.0,0.055405,0.058083,NaN,0.500000,0.416667,0.7964,0.057,...,-0.339285,0.177451,-0.033658,-0.076011,0.196520,-0.187527,-0.016569,-0.053284,0.065623,0.041845


In [96]:
lis=list(df_final.isnull().sum())
lis

[0,
 0,
 9,
 9,
 9,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [97]:
df_final=df_final.dropna()
df_final = df_final.drop(df_final.index[3734])
df_final = df_final.drop(df_final.index[4507])
df_final = df_final.drop(df_final.index[6610])
df_final = df_final.drop(df_final.index[8543])
df_final = df_final.drop(df_final.index[9150])
df_final = df_final.drop(df_final.index[9969])
df_final = df_final.drop(df_final.index[10287])
df_final = df_final.drop(df_final.index[11376])
df_final = df_final.drop(df_final.index[15277])

In [98]:
df_final.drop(df_final.tail(8).index,inplace = True)

In [99]:
df_final_null=df_final[df_final['have_email'].isnull()]

In [100]:
df_final_null

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,...,290,291,292,293,294,295,296,297,298,299


In [102]:
df_final.isnull().sum().sum()

0

In [103]:
df_final

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.027590,1.0,0.041849,0.051988,0.0,0.288095,-0.066667,-0.5358,0.116,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,0.000000,0.024992,0.0,0.043935,0.043551,1.0,0.631250,0.466667,0.9738,0.084,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,0.041667,0.030564,0.0,0.031839,0.035129,0.0,0.267143,0.097143,-0.2732,0.165,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,0.104167,0.025340,0.0,0.088564,0.085705,0.0,0.100000,0.300000,0.8779,0.126,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,0.020833,0.037957,0.0,0.156482,0.175179,0.0,0.406944,-0.016865,0.6555,0.117,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16638,0.000000,0.033965,0.0,0.024470,0.022954,0.0,0.066667,0.000000,-0.3400,0.124,...,-0.334503,0.221111,0.085950,-0.065862,0.146675,-0.274262,-0.003270,-0.089642,-0.027953,0.098107
16639,0.020833,0.034073,0.0,0.064095,0.066039,0.0,0.526190,-0.285714,0.0041,0.155,...,-0.284360,0.141470,-0.063051,-0.063888,0.085879,-0.104032,0.000692,-0.133151,0.038900,0.002948
16640,0.041667,0.032198,0.0,0.084115,0.084309,0.0,0.400000,-0.050000,0.8184,0.028,...,-0.304178,0.267353,0.026208,-0.110553,0.126186,-0.153754,0.022236,-0.060139,0.080693,0.054018
16641,0.020833,0.028072,0.0,0.108168,0.107263,0.0,0.462500,0.012500,-0.5719,0.180,...,-0.153379,0.199173,-0.060521,-0.054404,0.234239,-0.240764,-0.098948,-0.031596,0.023390,0.021918


In [104]:
df_final = df_final.reset_index(drop=True)

In [105]:
df_final

,comments,views,have_email,responses_message_length,gunning_fog_index,binary_score,subjectivity,polarity,compound,negative,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.027590,1.0,0.041849,0.051988,0.0,0.288095,-0.066667,-0.5358,0.116,...,-0.232025,0.172564,-0.019498,-0.071767,0.080226,-0.056546,0.018328,-0.099792,-0.003335,-0.059390
1,0.000000,0.024992,0.0,0.043935,0.043551,1.0,0.631250,0.466667,0.9738,0.084,...,-0.386816,0.157306,0.105151,-0.067104,0.155300,-0.043871,0.134495,-0.006914,-0.065409,0.132393
2,0.041667,0.030564,0.0,0.031839,0.035129,0.0,0.267143,0.097143,-0.2732,0.165,...,-0.202684,0.183406,-0.126370,-0.021195,0.139450,-0.124611,-0.044554,-0.100797,-0.037959,0.119683
3,0.104167,0.025340,0.0,0.088564,0.085705,0.0,0.100000,0.300000,0.8779,0.126,...,-0.403390,0.291392,0.017987,-0.073933,0.192811,-0.254528,0.044989,-0.104438,0.071245,0.132887
4,0.020833,0.037957,0.0,0.156482,0.175179,0.0,0.406944,-0.016865,0.6555,0.117,...,-0.226692,0.178298,0.004081,-0.032037,0.105949,-0.144188,0.000093,-0.018621,0.013283,0.058932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16620,0.000000,0.033965,0.0,0.024470,0.022954,0.0,0.066667,0.000000,-0.3400,0.124,...,-0.334503,0.221111,0.085950,-0.065862,0.146675,-0.274262,-0.003270,-0.089642,-0.027953,0.098107
16621,0.020833,0.034073,0.0,0.064095,0.066039,0.0,0.526190,-0.285714,0.0041,0.155,...,-0.284360,0.141470,-0.063051,-0.063888,0.085879,-0.104032,0.000692,-0.133151,0.038900,0.002948
16622,0.041667,0.032198,0.0,0.084115,0.084309,0.0,0.400000,-0.050000,0.8184,0.028,...,-0.304178,0.267353,0.026208,-0.110553,0.126186,-0.153754,0.022236,-0.060139,0.080693,0.054018
16623,0.020833,0.028072,0.0,0.108168,0.107263,0.0,0.462500,0.012500,-0.5719,0.180,...,-0.153379,0.199173,-0.060521,-0.054404,0.234239,-0.240764,-0.098948,-0.031596,0.023390,0.021918


In [107]:
df_final.binary_score = df_final.binary_score.astype('int64')

In [109]:
X=df_final
X=np.array(X.drop(['binary_score'],1))
y=np.array(df_final['binary_score'])

In [110]:
X.shape

(16625, 312)

In [111]:
y.shape

(16625,)

In [112]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [113]:
smote=SMOTE()

In [114]:
train_x_smote,train_y_smote=smote.fit_resample(x_train,y_train)

In [115]:
train_x_smote.shape

(17372, 312)

In [116]:
train_y_smote.shape

(17372,)

In [117]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [118]:
model=LinearDiscriminantAnalysis().fit(train_x_smote,train_y_smote)

In [119]:
y_pred=model.predict(x_test)
y_pred

array([0, 0, 1, ..., 0, 0, 1], dtype=int64)

In [120]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.73      0.78      2197
           1       0.58      0.72      0.64      1128

    accuracy                           0.73      3325
   macro avg       0.71      0.73      0.71      3325
weighted avg       0.75      0.73      0.73      3325



In [121]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred)

0.7258350361711812

In [122]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

In [131]:
rand_clf = RandomForestClassifier(criterion= 'entropy',
 max_depth = 17,
 max_features = 'auto',
 min_samples_leaf = 7,
 min_samples_split= 2,
 n_estimators = 200,random_state=6)

In [132]:
model_rf=rand_clf.fit(train_x_smote,train_y_smote)

In [133]:
y_pred_rf = model_rf.predict(x_test)
rf_pred = [round(value) for value in y_pred_rf]
rf_pred_accu = accuracy_score(y_test,rf_pred)
rf_pred_accu

0.7545864661654136

In [134]:
print(classification_report(y_test,rf_pred))

              precision    recall  f1-score   support

           0       0.83      0.78      0.81      2197
           1       0.62      0.70      0.66      1128

    accuracy                           0.75      3325
   macro avg       0.73      0.74      0.73      3325
weighted avg       0.76      0.75      0.76      3325



In [135]:
roc_auc_score(y_test,rf_pred)

0.7403142018290576

In [ ]:
# from sklearn.model_selection import train_test_split,GridSearchCV

In [ ]:
# grid_param = {
#     "n_estimators" : [90],
#     'criterion': ['gini','entropy'],
#     'max_depth' : range(2,20,3),
#     'min_samples_leaf' : range(1,10,3),
#     'min_samples_split': range(2,10,3),
#     'max_features' : ['auto','log2']
# }

In [ ]:
# grid_search = GridSearchCV(estimator=rand_clf,param_grid=grid_param,cv=3,n_jobs =1,verbose = 2)

In [ ]:
# grid_search.fit(train_x_smote,train_y_smote)

In [ ]:
# grid_search.best_params_